# Abstract
{Section to be edited}
>We have selected Project # 10 and are interested in learning how to simulate the airport passenger waiting time using SimPy, a process-based discrete-event simulation framework that runs in standard Python environment, and compare it with the simulation process using ARENA software. [Insert summary of the simulation model and major findings and conclusions]

# Table of Contents


# Description of Problem
Our task is to learn and demonstrate the capabilities of another high-level programming language. For this project, we have decided on PySim, an opensource simulation library within the Python language. While using a code-based simulation tool versus a graphical based one has it's challenges, we aim to make a simple to follow guide to getting started with PySim. Additionally, we will provide a few workable examples to demonstrate the power of SimPy.

# Main Findings

## Setting up the Environment

Because SimPy is based in python, it requires a slightly more in-depth approach to installation versus using a simple executable. We will not be covering the installation of python, if you need more help [check out it's documentation here](https://wiki.python.org/moin/BeginnersGuide/Download).  

SimPy runs on Python 3 or higher, though we recommend installing the latest version for more functionality. Once you have your python installation up and running, you can install Simpy easily like any other package through pip, just use the following:  
```pip install simpy```  
To confirm SimPy has been installed, you can use `pip list` to view the list of installed python packages

Now that we have SimPy, there may be some additional libraries you'd want to add. Namely, `numpy` or `scipy` to handle random number generation, which can be added the same way we installed simpy!


## Generating Random Numbers
SimPy doesn't offer a way to generate random numbers and because of this, users have to do this manually with libraries like `random`, `numpy` or `scipy`. We won't explore `scipy`, but let's go ahead and explore what random variables look like with the Python Standard Library `random` and `numpy`.

### Python Standard Library: Random
The Python Standard Library `random` is a built-in module that comes with Python. The module has a variety of pseudo-random number generators for a wide range of distributions. To learn more about this module, [check out the documentation](https://docs.python.org/3/library/random.html).

```python
# Load in the library
import random

# Define a seed value so numbers are repeatable
random.seed(123)

# PRN from 0 to 1
random.random()

# Uniform Distribution from 0, 1
random.uniform(0, 1)

# Exponential Distribution with a mean of 5
random.expovariate(1/5)
```

### Numpy
Numpy is a versatile Python library used for a lot of array and mathematical manipulations. Like `random` though, the library contains pseudo-random number generators for many distributions. To learn more about this module, [check out the documentation](https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html).

```python
# Load in the library
import numpy as np

# Define a seed value so numbers are repeatable
np.random.seed(123)

# PRN from 0 to 1
np.random.rand()

# Uniform Distribution from 0, 1
np.random.uniform(0, 1)

# Exponential Distribution with a lambda of 5
np.random.exponential(5)
```

## The Language  
### The basics of SimPy
Created by Kalus G.Muller and Tony Vignaux and released in September 2002, SimPy is a discrete-event Python framework used for event simulation by creating a virtual environment to reflect a real-world system. This can range from simulating queueing lines at a store, virus spread through a population, a manufacturing floor, or an airport terminal. The limits to what you can simulate are only limited by the knowledge and experience of the user!  

Because SimPy is a descrete-event simulation library, it will be necessary to program every component of our simulation including `processes` and `events`. `Processes` represent "customers" in our environment, while `events` show how their interactions and how they affect the environment. We'll explore the full utilization of `processes`,`events`, and the `environment` in the User Guide section.

As a quick 3 point overview, SimPy requires three major steps:
    1. Establish the environment  
    2. Pass in the parameters  
    3. Run the simulation




## User Guide

### Tutorial


### Examples

#### Airport Simulation 
Our airport waiting time simulation model contains a variety of global variables that can be modified to manipulate the simulation.

|Variable|Description|Type|Distribution|
|:----------------------|:------------------------------|--------|---|
|BOARDING_CHECK_WORKERS |# of Workers at Boarding Check|Resource||
|PERSONAL_CHECK_SCANNERS|# of Personal Scanners        |Resource||
|PASSENGERS_PER_MINUTE  |$\lambda$ for Passenger Arrival Rate|Parameter|Exponential|
|BOARDING_CHECK_SERVICE_RATE|$\lambda$ for Service Rate indicated boarding checkers|Parameter|Exponential|
|PERSONAL_CHECK_SERVICE_RATE|Service Rate min/max|Parameter|Uniform|
|RUN_TIME_MINUTES|Run time in minutes for each simulation|Parameter|
|REPLICATIONS|# of times to repeat the simulation|Parameter|

We can then change these variables to see how our simulation is affected and try to minimize the wait times of our passengers. We've set a variety of default inputs in our model, feel free to use the script and adjust as desired! 

Additonally, we initialize various lists to track simulation output across the runs. These include:
* `boarding_check_wait_times`|`personal_check_wait_times`
* `boarding_check_service_times`|`personal_check_service_times`
* `time_in_system`

The basic flow of our model is as follows: 
    1. Passenger enters the system
    2. Passenger flows through Boarding Checker
    3. System determines which Personal Check queue has shortest line
    4. Passenger flows through shortest Personal Check Queue
    5. Passenger exits the system
    6. Simulation records various statistics to provide analysis

This simulation continues for the duration of `RUN_TIME_MINUTES`, then repeats the number of `REPLICATIONS`.


Our main simulation program contains two classes: Airport and Passenger. These classes contain the various functions utilized while running the simulation. Utilizing Object-Oriented-Programming in this way with python classes is the easiest way to create simulations with SimPy. For some detailed material on Python OOP, [Check out this Real Python Tutorial](https://realpython.com/python3-object-oriented-programming/).

In our Airport Class, we have defined the following functions:  
* `__init__(self,env)`  
    * env represents simpy.Environment() which initializes our simulation environment
    * `self.boarding_check` creates a resource in our environment with capacity `BOARDING_CHECK_WORKERS`
    * `self.personal_check_scanner` creates X number of personal_check_scanner resources, each with their own queue.
* `boarding_check_service_time(self)`
    * Defines the `boarding_check_service_time` with yield
    * Event is distributed ~Exponential(`BOARDING_CHECK_SERVICE_RATE`)
* `personal_check_service_time(self)`
    * Defines the `personal_check_service_time` with yield
    * Event is distributed ~Uniform(Min Personal Check Service Rate, Max Personal Check Service Rate)
* `get_resources(self)`
    * Creates and returns resource dictionary for our boarding_check and personal_check resources
    * Dictionary contains information on overall simulation wait times, service times, and the resource serice rate

In our Passenger Class, we have defined the following functions:
* `__init__(self, name, airport)`
    * Passenger name will be generated in a loop, representing which # passenger is currently going through the simulation
    * Airport will be fed in from the airport class
    * Creates an arrival time variable by determining the current simulation time when the passenger arrives
* `go_to_airport(self)`
    1. Sends Passengers through the boarding_check line
    2. Runs the decision block to determine personal_check_scanner with the shorters queue
    3. Sends Passenger through the shortest available personal_check_scanner
* `_create_process_dispose(self, resource, name, arrival_time)`
    * This is where we Create, Process, and Dispose of passengers through each event
    * Takes in a resource from the aiport class dictionary, passenger name, and passenger arrival time
    * Utilizes resource.request() to create entities, store their wait times, process them, and dispose automatically on finish
* `_decision_block(self)`
    * Utilizes a for loop and if statement to determine which personal_check_scanner has the shortest line

Now that we have our classes created, we define a function to generate our passengers:

`passenger_generator(env)`:
* This will generate passengers for the duration of our simulation
* Passengers are distributed ~Exponential(1/`PASSENGERS_PER_MINUTE`)
* Creates numbered passengers, and an airport environment. Then feeds each passenger and airport to the passenger class
* `env.process(passenger.go_to_airport())` sets up the simulation environment to run for each passenger until simulation ends

Finally, we utilize a for loop to loop through each `REPLICATIONS` and actually run our simulation now that everything is initialized. This can be seen in the following code snippet:  
```for i in range(1, REPLICATIONS + 1):
    np.random.seed(i)
    env = simpy.Environment()
    env.process(passenger_generator(env))
    env.run(until=RUN_TIME_MINUTES)```
* We change the random seed on each loop, to ensure each replication isn't identical
* `env` intitializes the Simpy Environment
* `Env.process` tells the environment to begin the passenger_generator event
* `env.run` tells our simulation our long to run, in this case each replication will run for `RUN_TIME_MINUTES`

On each loop, we are storing various simulation statistics for output analysis, which is handled in a neat table format utilizing tabulate. 

This brings us to the end of our Airport simulation. With this simulation, we are able to modify various variables to tweak our simulation and see how that affects outputs such as wait times, average service rates, and average time in the system. 

For a more visual representation of this example, this airport simulation in ARENA would look like the following:
![airport-arena-image.png](https://i.imgur.com/yUbzMAs.png)

## Comparison to Arena
#### The UI
When comparing Pysim to Arena, the first obvious note is that Pysim is code based while Arena has a flowchart based GUI. This makes Arena much more approachable for those without a coding background, such as within a business setting. Having a GUI only interface can be useful as it's easy to explore visually and find different modules while building the simulation, though it can be clunky and overwhelming. Despite this, Arena can be grasped very quickly as users navigate around the UI, reading different modules and drag+dropping their utilities. This allows simulations in arena to be built quickly and easily for anyone with a basic understanding of statistics and simulations. Contrastly Simpy's code-based approach can be intimidating to non-programmers. While it allows for easy integration into advanced models and workflows, this is heavily dependent on the developers coding abilities. 

#### Interactivity & Demo ability
A major advantage Arena has over SimPy is it's interactivity. The ability to drag&drop modules, live run the simulation, then view customers passing between blocks is a great benefit when trying to model a demo of a simulation for clients. There is the potential to code this up from scratch in python by utilizing many external libraries, but doing so is the equivalent of riding a bike across the country instead of driving a car. Yes you CAN, but it's much more efficient to use the proper tool for the job.

#### Cost
There's not much competition from the cost perspective. Arena's professional use pricing model is expesive, with pricing information difficult to find. The most recent prices we have found put an Arena license at roughly \\$2,500 USD for a basic license, and \\$19,500 USD for a professional license. These prices were pulled from a research paper in 2011, as Arena does not make it's pricing model public. Contrastly Pysim is free for both personal and professional use, making it much more approachable for new project exploration. 

### Pros
 - Code-based
 - Free
 - Full interactivity with other python packages
 - Easily integrate within other models and data pipelines
 - Open Source

### Cons
- Code-based, Steep learning curve for non-programmers
- Relies on outside packages for things such as random number generation, charting, etc.
- Not easily interactive or visual, Can't easily create visual simulation demo's
- Limited to the developers skill level

# Conclusions
>Both SimPy simulation and ARENA Queueing Model could improve the airport queueing system and therefore increases the efficiency of airline operations, by reducing the waiting time via optimizing the # check-in stations and check-in workers.
While ARENA provides graphical simulation modeling and modules templates that is visually easier to navigate and animate than the code-based SimPy, it is also far more costly than its non-proprietary counterpart which is free and open to everyone.On ther hand, SimPy requires certain Python programming aptitude in order to successfully carry out the simulation and therefore may require certain learning time beforehand. [Insert other key pts from the simulation model and a concluding sentence]

# Appendix
#### Resources
- https://realpython.com/simpy-simulating-with-python/#how-simulation-works
- https://simpy.readthedocs.io/
- https://realpython.com/python3-object-oriented-programming/
